In [57]:
import os
from sklearn.model_selection import train_test_split
import gensim
import gensim.downloader as api
from sklearn.neighbors._classification import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import urllib.request, json
from nltk.tokenize import sent_tokenize, word_tokenize
import random

### Load the Data

In [2]:
jb_folder = '/Users/romitbarua/Documents/Berkeley/Spring 2023/world_leaders/combined/worldleaders20/jb'
br_folder = '/br'

In [3]:
def processWord(word):
    word = word.lower()
    return ''.join(e for e in word if e.isalnum())
    

In [4]:
path = '/Users/romitbarua/Documents/Berkeley/Spring 2023/world_leaders/combined/worldleaders20'
speakers = ['jb', 'br', 'pb', 'kh', 'cb']

data = []
data_tags = []

for speaker in speakers:
    for file in os.listdir(os.path.join(path,speaker)):
        file_content = []
        if ('.txt' in file) and ('phone' not in file):
            with open(os.path.join(path, speaker,file)) as f:
                lines = f.readlines()
                for line in lines:
                    
                    file_content.append(processWord(line.split(',')[0]))
                    
            data.append(file_content)
            data_tags.append(speaker)
        
        

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, data_tags, test_size=0.10)

In [6]:
print(f'Number of training documents: {len(X_train)}')
print(f'Number of testing documents: {len(X_test)}')

Number of training documents: 1117
Number of testing documents: 125


In [7]:
#pip install numpy --upgrade

In [8]:
#dataset = api.load("text8")

In [9]:
#data = [d for d in dataset]

In [10]:
def tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])

In [11]:
data_training = list(tagged_document(X_train))

In [12]:
data_training [:2]

[TaggedDocument(words=['not', 'just', 'tactically', 'focusing', 'in', 'on', 'him', 'we', 'cannot', 'run', 'this', 'race', 'about', 'what', 'were', 'against', 'we', 'have', 'to', 'run', 'this', 'race', 'by', 'what', 'were', 'four', 'because', 'thats', 'going', 'to', 'inspire', 'people', 'thats', 'going', 'to', 'energize', 'folks', 'at', 'the', 'tactics', 'of', 'just', 'beating', 'him', 'im', 'sorry', 'he', 'wants', 'to', 'make', 'this', 'all', 'about', 'him', 'he', 'wants', 'to', 'suck', 'all', 'the', 'oxygen', 'again', 'out', 'of', 'an', 'election', 'and', 'he', 'wants', 'us', 'to', 'fight', 'him', 'on', 'his', 'turfs', 'and', 'his', 'terms', 'i', 'know', 'this', 'from', 'going', 'up', 'against', 'tough'], tags=[0]),
 TaggedDocument(words=['theyre', 'wondering', 'who', 'it', 'is', 'that', 'ted', 'cruz', 'represent', 'in', 'the', 'senate', 'hes', 'taken', 'hundreds', 'of', 'thousands', 'of', 'dollars', 'from', 'the', 'nra', 'so', 'as', 'people', 'are', 'begging', 'us', 'to', 'take', 'ac

In [13]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=75, min_count=3, epochs=800)

In [14]:
model.build_vocab(data_training)

In [15]:
print(model.infer_vector(['violent', 'means', 'to', 'destroy', 'the','organization']))

[-0.0018015  -0.00469368 -0.00369109 -0.0012169  -0.00447437 -0.00347355
 -0.00558665 -0.00653968  0.00551053 -0.00426685 -0.00238561  0.00063201
  0.00154458 -0.00454299 -0.00551792  0.00224212  0.00240443  0.00400222
  0.00434341  0.00066305  0.00506318 -0.00409319 -0.0034704  -0.00317483
 -0.00475692 -0.00579562  0.0033451   0.0014919  -0.00486575  0.00061452
  0.0029611  -0.00617157 -0.00475545 -0.00550879  0.006036   -0.00412667
 -0.00271706 -0.00076478  0.0027844  -0.00151195  0.00035146 -0.00239385
 -0.00072916 -0.00333275  0.00608678 -0.00418608  0.00323968  0.00264291
 -0.00051592 -0.00282541 -0.00535957 -0.00017715 -0.00498864 -0.00041258
 -0.00509849  0.00506417  0.00323221  0.00188181 -0.00157215 -0.0020488
  0.00183242  0.00506284 -0.00150484 -0.00163814 -0.00597288  0.00433506
 -0.00478104 -0.00363955  0.00510365  0.00418079 -0.0011654  -0.00494329
  0.00608666 -0.00347432  0.00340798]


In [16]:
train_embeddings = []

for i in X_train:
    train_embeddings.append(model.infer_vector(i))

In [17]:
test_embeddings = []

for i in X_test:
    test_embeddings.append(model.infer_vector(i))

In [18]:
knn = KNeighborsClassifier()

In [19]:
knn.fit(train_embeddings, y_train)

KNeighborsClassifier()

In [20]:
pred = knn.predict(test_embeddings)
acc = accuracy_score(y_test, pred)

/Users/romitbarua/opt/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [21]:
print(acc)

0.224


# Full Presidential Speech Data

In [70]:
with urllib.request.urlopen('https://millercenter.org/sites/default/files/corpus/presidential-speeches.json') as url:
    data = json.load(url)
    print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [73]:
obama_speeches = [] #Barack Obama
bush_speeches = [] #George W. Bush

for i in data:

    if i['president'] == 'Barack Obama':
        obama_speeches.append(i['transcript'])
    elif i['president'] == 'George W. Bush':
        bush_speeches.append(i['transcript'])
        

In [74]:
obama_sent = []
for speech in obama_speeches:
    obama_sent.extend(sent_tokenize(speech))
len(obama_sent)

12005

In [75]:
bush_sent = []
for speech in bush_speeches:
    bush_sent.extend(sent_tokenize(speech))
len(bush_sent)

6475

In [76]:
obama_token_sent = []
for sent in obama_sent:
    obama_token_sent.append(word_tokenize(sent))
    
bush_token_sent = []
for sent in bush_sent:
    bush_token_sent.append(word_tokenize(sent))
    
random.shuffle(obama_token_sent)
random.shuffle(bush_token_sent)

In [77]:
train_perc = 0.8

obama_train = obama_token_sent[:int(len(obama_token_sent)*train_perc)]
bush_train = bush_token_sent[:int(len(bush_token_sent)*train_perc)]
obama_test = obama_token_sent[int(len(obama_token_sent)*train_perc):]
bush_test = bush_token_sent[int(len(bush_token_sent)*train_perc):]

X_train = obama_train + bush_train
X_test = obama_test + bush_test
y_train = [1]*len(obama_train) + [0]*len(bush_train)
y_test = [1]*len(obama_test) + [0]*len(bush_test)

In [78]:
data_training = list(tagged_document(X_train))

In [79]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=3, epochs=100)
model.build_vocab(data_training)

In [80]:
train_embeddings = []

for i in X_train:
    train_embeddings.append(model.infer_vector(i))

In [81]:
test_embeddings = []

for i in X_test:
    test_embeddings.append(model.infer_vector(i))

In [82]:
knn = KNeighborsClassifier()
knn.fit(train_embeddings, y_train)

KNeighborsClassifier()

In [83]:
pred = knn.predict(test_embeddings)
acc = accuracy_score(y_test, pred)
print(acc)

0.5906385281385281


/Users/romitbarua/opt/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
